In [9]:
import io
import pandas as pd
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

## -- Variables & connections
username = ''
password = ''
url = ''
#funciones
def connect_to_sharepoint(url_file):
   ctx_auth = AuthenticationContext(url_file)
   if ctx_auth.acquire_token_for_user(username, password):
       ctx = ClientContext(url_file, ctx_auth)
       web = ctx.web
       ctx.load(web)
       ctx.execute_query()
   response = File.open_binary(ctx, url_file)
   bytes_file_obj = io.BytesIO()
   bytes_file_obj.write(response.content)
   bytes_file_obj.seek(0)
   return(bytes_file_obj)
#body
df = pd.read_excel(connect_to_sharepoint(url), header=0)

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import os

#Sharepoint Variables
site_url = 'https://company.sharepoint.com/sites/site_name'
username = ''
password = "" 
folder_url_destination = '/sites/'
#Local Variables
local_directory = r'C:\Users\username\folder_name' 
file_name = 'file_name' + '.xlsx'
source_path = os.path.join(local_directory, file_name)

ctx_auth = AuthenticationContext(site_url)
if ctx_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(site_url, ctx_auth)
    folder = ctx.web.get_folder_by_server_relative_url(folder_url_destination)  

    with open(source_path, 'rb') as file_content:
        file_name = source_path.split(f'\\')[-1]
        file_creation_info = {
            'FileLeafRef': file_name,
            'Content': file_content.read()
        }
    #print(f'File {file_name} ')
    # Upload the file
    upload_file = folder.upload_file(file_name, file_creation_info['Content']).execute_query()
    print(f'*** - File {file_name} has been uploaded to {folder_url_destination}')
else:
    print('Authentication failed')

In [10]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import os
import datetime
import time
import re
import pandas as pd
import io

#Sharepoint Variables
site_url = 'https://companyname.sharepoint.com/sites/site_name'
folder_url = '/sites/Active/'         
username = ''
password = ""
pattern = r"_([^\.]+)\."

#--------------------------------------------
#               Functions
#--------------------------------------------

def get_folder_files():
    ctx_auth = AuthenticationContext(site_url)

    if ctx_auth.acquire_token_for_user(username, password):
        ctx = ClientContext(site_url, ctx_auth)

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        ctx.load(folder)
        ctx.execute_query()

        files = folder.files
        ctx.load(files)
        ctx.execute_query()

        #Columnas
        df_name = []
        df_date = []
        df_user = []
        df_uniqueid = []
        df_relativeurl = []
        df_length = []

        # Download files to local directory
        #os.makedirs(local_directory, exist_ok=True)

        for file in files:
            file_name = file.properties['Name']
            if not file_name.startswith('~tmp'):
                df_name.append(file_name)
                match = re.search(pattern, file_name)
                if match:
                    name = match.group(1)
                    df_user.append(name.split()[0] + ' ' + name.split()[1])
                df_date.append(file.properties['TimeCreated'] - datetime.timedelta(hours=5)) #.replace(second=0, microsecond=0))
                df_uniqueid.append(file.properties['UniqueId'])
                df_relativeurl.append(file.properties['ServerRelativeUrl'])
                df_length.append(str(round(int(file.properties['Length']) / 1024, 0)) + ' KB')

        if len(df_name) == len(df_date) == len(df_uniqueid) == len(df_relativeurl) == len(df_length):
            # Create a dictionary
            data = {
                'File_Name': df_name,
                'User': df_user,
                'Date': df_date,
                'UniqueID': df_uniqueid,
                'RelativeURL': df_relativeurl,
                'Length_bytes': df_length
            }
    return pd.DataFrame(data)

#--------------------------------------------
#               Process Init
#--------------------------------------------

df = get_folder_files()
df

,File_Name,User,Date,UniqueID,RelativeURL,Length_bytes
0,PANAMEÑO ALCANZADO AL MUNDO - SPN LINEA 6272-_...,Yenny Lorena,2024-06-06 09:39:10,d2e9444f-401f-4583-9e57-002bcab92a42,/sites/ActivacionesPDF/Shared Documents/Apps/M...,306.0 KB
1,pag 2 junio 24_Zabdy Hernandez.png,Zabdy Hernandez,2024-06-18 13:19:51,f96ea3d4-0d6e-4538-87cf-0080cdd1de20,/sites/ActivacionesPDF/Shared Documents/Apps/M...,1691.0 KB
2,PAGO FACT JESSICA DE LEON JU24_Natalia Gallard...,Natalia Gallardo,2024-06-20 06:24:28,8ed2b192-ac8c-4a71-bc47-01eef078060c,/sites/ActivacionesPDF/Shared Documents/Apps/M...,394.0 KB
3,MINUTA_Francisco Suarez 2.pdf,Francisco Suarez,2024-06-03 17:46:13,7cdbb761-097d-4970-adc9-01f43afab3f9,/sites/ActivacionesPDF/Shared Documents/Apps/M...,99.0 KB
4,REGISTRO PÚBLICO_Zabdy Hernandez.pdf,Zabdy Hernandez,2024-06-28 14:50:24,a8a700af-ad4e-41b9-a470-0205d6976a25,/sites/ActivacionesPDF/Shared Documents/Apps/M...,236.0 KB
...,...,...,...,...,...,...
216,kam 2_Melanie Weeks.pdf,Melanie Weeks,2024-06-11 09:33:06,272bb7f3-3e47-48ae-adc2-fc368b5f7569,/sites/ActivacionesPDF/Shared Documents/Apps/M...,67.0 KB
217,Registro Publico nuevo_Zabdy Hernandez.pdf,Zabdy Hernandez,2024-06-27 16:47:44,004e16a3-f5b9-4342-8cb4-fcf16eadb542,/sites/ActivacionesPDF/Shared Documents/Apps/M...,456.0 KB
218,CARMEN LAW JUAREZ - PAGO DE LINEA 62801667_Fra...,Francisco Suarez,2024-05-29 09:59:18,23c60af8-7d97-44c1-af08-fe0b976b2ce5,/sites/ActivacionesPDF/Shared Documents/Apps/M...,6.0 KB
219,FORM PORTA JESSICA DE LEON_Natalia Gallardo.pdf,Natalia Gallardo,2024-06-20 06:24:28,64254265-375f-4f70-9adb-fe760da1fd3a,/sites/ActivacionesPDF/Shared Documents/Apps/M...,466.0 KB
